# CNN para 33 actividades

## Separación de sets
Primero separamos los quaternions en sets de entrenamiento, validación y test:

In [1]:
import os as os

In [2]:
import tensorflow.keras as keras

In [3]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import sys
from six.moves import cPickle as pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [4]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [5]:
keras.__version__

'2.2.4-tf'

# Generación del dataset

Hacemos uso de los ficheros que contiene los cuaterniones separados por sujetos y actividad.

Definimos una serie de funciones para la lectura de los datos y finalmente realizamos el dataset (así leemos los datos en el entrenamiento "al vuelo").

In [6]:
# Ruta raíz
PATH = '../data_reset_def'

# Ruta de los ceckpoints
CKPATH = PATH + '/checkpoints'

quaturls = !ls -1 "{PATH}"

In [7]:
global activities
activities =  [19]
global subjects
#subjects = [1,2,3,5,8,9,10,11,13,14,16,17]
subjects = [1]

In [8]:
quat_corr = []
for filename in quaturls:
    for activity in activities:
        if(int(filename[-6:-4])==activity):
            quat_corr.append(filename)

In [9]:
quat_def = []

i=0
for filename in quat_corr:
    for subject in subjects:
        if(int(quat_corr[i][8:10])==subject):
            quat_def.append(filename)
    i+=1

In [10]:
quaturls = quat_def

del quat_corr, quat_def

#print(quaturls)

### Separación por sujetos

In [11]:
# Shuffle de los datos
n = len(quaturls)

tr_urls = []
va_urls = []
ts_urls = []

i=0
for filename in quaturls:
    if(int(quaturls[i][8:10])==2 or int(quaturls[i][8:10])==10):
        va_urls.append(filename)
    elif(int(quaturls[i][8:10])==16):
        ts_urls.append(filename)
    else:
        tr_urls.append(filename)
    i+=1

print('Total files: ' + str(len(quaturls)))
print('Train files: ' + str(len(tr_urls)))
#print('Validation files: ' + str(len(va_urls)))
#print('Test files: ' + str(len(ts_urls)))

Total files: 1
Train files: 1


In [12]:
tr_fullpath = [os.path.join(PATH,s) for s in tr_urls]
print('TRAINING: ')
print(tr_fullpath)
'''
va_fullpath = [os.path.join(PATH,s) for s in va_urls]
print('VALIDATION: ')
print(va_fullpath)
ts_fullpath = [os.path.join(PATH,s) for s in ts_urls]
print('TEST: ')
print(ts_fullpath)
'''

TRAINING: 
['../data_reset_def/subject_01_RESET_act_19.csv']


"\nva_fullpath = [os.path.join(PATH,s) for s in va_urls]\nprint('VALIDATION: ')\nprint(va_fullpath)\nts_fullpath = [os.path.join(PATH,s) for s in ts_urls]\nprint('TEST: ')\nprint(ts_fullpath)\n"

### Con listas y numpy (la más pesada de todas, pero seguros en que mantiene la estructura de los datos)

In [13]:
def sliding_window(df, n_time_steps, step, segments, labels, label, dim, train, n_channels):
    quat0 = df.iloc[:, 1:5][df['QUAT']=='quat0'].reset_index() # si no incluimos el reset_index(), al concatenarlos después
    quat1 = df.iloc[:, 1:5][df['QUAT']=='quat1'].reset_index() # aparecerá un dataframe de igual nº de filas que df, pero con
    quat2 = df.iloc[:, 1:5][df['QUAT']=='quat2'].reset_index() # NaN en las posiciones que no tienen número de cada dataframe quat,
    quat3 = df.iloc[:, 1:5][df['QUAT']=='quat3'].reset_index() # es decir, mantiene los índices de df.
    quat4 = df.iloc[:, 1:5][df['QUAT']=='quat4'].reset_index()
    #quat5 = df.iloc[:, 1:5][df['QUAT']=='quat5'].reset_index()
    #quat6 = df.iloc[:, 1:5][df['QUAT']=='quat6'].reset_index()
    #quat7 = df.iloc[:, 1:5][df['QUAT']=='quat7'].reset_index()
    #quat8 = df.iloc[:, 1:5][df['QUAT']=='quat8'].reset_index()
    
    quat = pd.concat([quat0.iloc[:, 1], quat0.iloc[:, 2], quat0.iloc[:, 3], quat0.iloc[:, 4],
                      quat1.iloc[:, 1], quat1.iloc[:, 2], quat1.iloc[:, 3], quat1.iloc[:, 4],
                      quat2.iloc[:, 1], quat2.iloc[:, 2], quat2.iloc[:, 3], quat2.iloc[:, 4],
                      quat3.iloc[:, 1], quat3.iloc[:, 2], quat3.iloc[:, 3], quat3.iloc[:, 4],
                      quat4.iloc[:, 1], quat4.iloc[:, 2], quat4.iloc[:, 3], quat4.iloc[:, 4]],
                      #quat5.iloc[:, 1], quat5.iloc[:, 2], quat5.iloc[:, 3], quat5.iloc[:, 4],
                      #quat6.iloc[:, 1], quat6.iloc[:, 2], quat6.iloc[:, 3], quat6.iloc[:, 4],
                      #quat7.iloc[:, 1], quat7.iloc[:, 2], quat7.iloc[:, 3], quat7.iloc[:, 4],
                      #quat8.iloc[:, 1], quat8.iloc[:, 2], quat8.iloc[:, 3], quat8.iloc[:, 4]],
                      axis = 1, keys = ['w0', 'x0', 'y0', 'z0', 'w1', 'x1', 'y1', 'z1', 'w2', 'x2', 'y2', 'z2', 
                                        'w3', 'x3', 'y3', 'z3', 'w4', 'x4', 'y4', 'z4']) #'w5', 'x5', 'y5', 'z5',
                                        #'w6', 'x6', 'y6', 'z6', 'w7', 'x7', 'y7', 'z7', 'w8', 'x8', 'y8', 'z8'])
    del quat0, quat1, quat2, quat3, quat4 #quat5, quat6, quat7, quat8
    
    if(train==False):
        step = n_time_steps
    
    if(n_channels == 1):
        for i in range(0, quat.shape[0] - n_time_steps, step): # Overlap
            # Con listas y numpy
            segments.append([])
            segments[dim].append(quat.iloc[i: i + n_time_steps, :].values) # Si distinguimos entre sensores
            labels.append(label-1)
            dim+=1
    else:
        n_columns = int(36/n_channels)
        for i in range(0, quat.shape[0] - n_time_steps, step):
            segments.append([])
            col = 0
            for j in range(n_channels):
                segments[dim].append([])
                segments[dim][j].append(quat.iloc[i:i+n_time_steps,col:col+n_columns].values)
                col+=n_columns
            labels.append(label-1)
            dim+=1
    
        
    del quat
    
    return segments, labels, dim

In [14]:
def load_quat(path, train, n_channels):

    n_time_steps, step = 1, 1  # n_time_steps/50 segundos de actividad y pasos de step/50 segundos (de overlap)
    
    # Con listas
    segments = []
    labels = []
    
    i=1
    dim = 0
    for filename in path:
        print("Reading %s (%d/%d)                                                   "%(filename, i, len(path)), end='\r')
        df = pd.read_csv(filename,sep=',')
        label = int(filename[-6:-4])
        for i in range(len(activities)):
            if(label==activities[i]):
                label=i+1
        
        segments, labels, dim = sliding_window(df, n_time_steps, step, segments, labels, label, dim, train, n_channels)
        
        i+=1
    
    del df
    
    return segments, labels


def load_train_quat(filename, n_channels):
    return load_quat(filename, True, n_channels)

def load_test_quat(filename, n_channels):
    return load_quat(filename, False, n_channels)


In [15]:
def get_dataset(data_path, batch_size, n_time_steps, train, valid, ds, n_channels):
    if(train):
        segments, labels = load_train_quat(data_path, n_channels)
    else:
        segments, labels = load_test_quat(data_path, n_channels)
    
    print('Generating the dataset                                                   ')                                              
    
    array = np.asarray(segments, dtype = 'float32')
    segments = np.reshape(array, (array.shape[0], n_channels, n_time_steps, int(20/n_channels)))
    array = np.asarray(labels, dtype = 'int8')
    labels = np.reshape(array, (array.shape[0], 1))
    
    del array
    
    # Map coninous dataset to categorical (One-Hot)
    labels = keras.utils.to_categorical(labels, 17)
    
    if(train):
        print('-'*20 + 'TRAIN' + '-'*20)
    elif(valid):
        print('-'*18 + 'VALIDATION' + '-'*12)
    else:
        print('-'*20 + 'TEST' + '-'*21)
    
    if(ds):
        dataset = tf.data.Dataset.from_tensor_slices((segments, labels))
    
        # It's necessary to repeat our data for all epochs
        dataset = dataset.batch(batch_size)
        
        dataset = dataset.shuffle(segments.shape[0])
        
        print('Dataset generated                                                        ')
        
        return dataset, segments, labels
    else:
        # Shuffle in the first dimension
        #permutation = np.arange(0,segments.shape[0]-1)
        #np.random.shuffle(permutation)
        #segments = segments[permutation]
        #labels = labels[permutation]
        print('Dataset generated                                                        ')
        
        return segments, labels

In [16]:
n_time_steps = 1
dataset = False # Set to True if you want a dataset or to False if you want np.arrays
batch_size = 1 # REAL batch_size
n_channels = 1 # It can be 1,4 or 9

if(dataset):
    train_dataset, tr_seg, tr_lab = get_dataset(tr_fullpath, ind, batch_size, n_time_steps, True, False, dataset, n_channels)
    valid_dataset, va_seg, va_lab = get_dataset(va_fullpath, ind, batch_size, n_time_steps, False, True, dataset, n_channels)
    test_dataset, ts_seg, ts_lab = get_dataset(ts_fullpath, ind, batch_size, n_time_steps, False, False, dataset, n_channels)
    
    print('Train dataset: ')
    print(train_dataset)
    print('Validation dataset: ')
    print(valid_dataset)
    print('Test dataset: ')
    print(test_dataset)
else:
    tr_seg, _ = get_dataset(tr_fullpath, batch_size, n_time_steps, True, False, dataset, n_channels)
    #va_seg, _ = get_dataset(va_fullpath, batch_size, n_time_steps, False, True, dataset, n_channels)
    #ts_seg, _ = get_dataset(ts_fullpath, batch_size, n_time_steps, False, False, dataset, n_channels)
    print('Train dataset: ')
    print(tr_seg.shape)
    '''
    print('Validation dataset: ')
    print(va_seg.shape)
    print('Test dataset: ')
    print(ts_seg.shape)
    '''

Generating the dataset                                                                                        
--------------------TRAIN--------------------
Dataset generated                                                        
Train dataset: 
(1231, 1, 1, 20)


In [17]:
tr_seg = np.reshape(tr_seg,(tr_seg.shape[0],tr_seg.shape[2],tr_seg.shape[3]))
'''
va_seg = np.reshape(va_seg,(va_seg.shape[0],va_seg.shape[2],va_seg.shape[3]))
ts_seg = np.reshape(ts_seg,(ts_seg.shape[0],ts_seg.shape[2],ts_seg.shape[3]))
'''

'\nva_seg = np.reshape(va_seg,(va_seg.shape[0],va_seg.shape[2],va_seg.shape[3]))\nts_seg = np.reshape(ts_seg,(ts_seg.shape[0],ts_seg.shape[2],ts_seg.shape[3]))\n'

In [18]:
print('Train dataset: ')
print(tr_seg.shape)
'''
print('Validation dataset: ')
print(va_seg.shape)
print('Test dataset: ')
print(ts_seg.shape)
'''

Train dataset: 
(1231, 1, 20)


"\nprint('Validation dataset: ')\nprint(va_seg.shape)\nprint('Test dataset: ')\nprint(ts_seg.shape)\n"

In [19]:
tr_seg = np.reshape(tr_seg,(tr_seg.shape[0],tr_seg.shape[2]))
'''
va_seg = np.reshape(va_seg,(va_seg.shape[0],va_seg.shape[2]))
ts_seg = np.reshape(ts_seg,(ts_seg.shape[0],ts_seg.shape[2]))
'''

'\nva_seg = np.reshape(va_seg,(va_seg.shape[0],va_seg.shape[2]))\nts_seg = np.reshape(ts_seg,(ts_seg.shape[0],ts_seg.shape[2]))\n'

In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
tr_seg = scaler.fit_transform(tr_seg)
'''
va_seg = scaler.fit_transform(va_seg)
ts_seg = scaler.fit_transform(ts_seg)
'''

'\nva_seg = scaler.fit_transform(va_seg)\nts_seg = scaler.fit_transform(ts_seg)\n'

In [21]:
tr_seg = np.reshape(tr_seg,(tr_seg.shape[0], 1, tr_seg.shape[1]))
'''
va_seg = np.reshape(va_seg,(va_seg.shape[0], 1, va_seg.shape[1]))
ts_seg = np.reshape(ts_seg,(ts_seg.shape[0], 1, ts_seg.shape[1]))
'''

'\nva_seg = np.reshape(va_seg,(va_seg.shape[0], 1, va_seg.shape[1]))\nts_seg = np.reshape(ts_seg,(ts_seg.shape[0], 1, ts_seg.shape[1]))\n'

In [22]:
def create_y(seg):
    Y = []
    for i in range(128, seg.shape[0]):
        Y.append(seg[i,0,:])
    return np.array(Y)

In [23]:
trainX = np.reshape(tr_seg[:-128],(tr_seg.shape[0]-128, tr_seg.shape[1], tr_seg.shape[2]))
#trainY = np.reshape(tr_seg[1:],(tr_seg.shape[0]-1, tr_seg.shape[1], tr_seg.shape[2]))
trainY = create_y(tr_seg)
'''
validX = np.reshape(va_seg[:-1],(va_seg.shape[0]-1, va_seg.shape[1], va_seg.shape[2]))
#validY = np.reshape(va_seg[1:],(va_seg.shape[0]-1, va_seg.shape[1], va_seg.shape[2]))
validY = create_y(va_seg)

testX = np.reshape(ts_seg[:-1],(ts_seg.shape[0]-1, ts_seg.shape[1], ts_seg.shape[2]))
#testY = np.reshape(ts_seg[1:],(ts_seg.shape[0]-1, ts_seg.shape[1], ts_seg.shape[2]))
testY = create_y(ts_seg)
'''

'\nvalidX = np.reshape(va_seg[:-1],(va_seg.shape[0]-1, va_seg.shape[1], va_seg.shape[2]))\n#validY = np.reshape(va_seg[1:],(va_seg.shape[0]-1, va_seg.shape[1], va_seg.shape[2]))\nvalidY = create_y(va_seg)\n\ntestX = np.reshape(ts_seg[:-1],(ts_seg.shape[0]-1, ts_seg.shape[1], ts_seg.shape[2]))\n#testY = np.reshape(ts_seg[1:],(ts_seg.shape[0]-1, ts_seg.shape[1], ts_seg.shape[2]))\ntestY = create_y(ts_seg)\n'

In [24]:
print(trainX.shape, trainY.shape)
'''
print(validX.shape, validY.shape)
print(testX.shape, testY.shape)
'''

(1103, 1, 20) (1103, 20)


'\nprint(validX.shape, validY.shape)\nprint(testX.shape, testY.shape)\n'

In [25]:
print(trainX[128])
print(trainY[0])

[[0.4357562  0.57302177 0.49610472 0.45991588 0.5193336  0.8978774
  0.6245612  0.13116932 0.47364688 0.8308481  0.66223705 0.05037677
  0.5109245  0.18340534 0.57968324 0.92990434 0.5778404  0.34200484
  0.4863946  0.08713397]]
[0.4357562  0.57302177 0.49610472 0.45991588 0.5193336  0.8978774
 0.6245612  0.13116932 0.47364688 0.8308481  0.66223705 0.05037677
 0.5109245  0.18340534 0.57968324 0.92990434 0.5778404  0.34200484
 0.4863946  0.08713397]


### Test

In [26]:
from tensorflow.keras.models import load_model
rnn = load_model('./00_06b_models/predict_rnn.h5')
# Compilamos el modelo
rnn.compile(optimizer=keras.optimizers.RMSprop(lr=1e-4), loss='mean_squared_error', metrics=['accuracy'])
print('RNN cargada y compilada.')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
RNN cargada y compilada.


In [45]:
initial=-1

In [46]:
tX = trainX[:initial]
print(tX.shape)

(1102, 1, 20)


In [47]:
t=1024
for i in range(t):
    print('%d/%d'%(i+1,t), end='\r')
    prediction=rnn.predict(np.reshape(tX[-128:],(128,1,20)))
    pred=np.reshape(prediction[-1],(1,1,prediction.shape[1]))
    tX=np.concatenate((tX,pred))
    

In [48]:
print(tX.shape)

(2126, 1, 20)


In [49]:
predict = tX[1102:]
print(predict.shape)

(1024, 1, 20)


In [50]:
predict=np.reshape(predict,(predict.shape[0],predict.shape[2]))

In [51]:
predict = scaler.inverse_transform(predict)

### Lo escribimos a fichero

In [52]:
def write(name, ts_seg):
    fo = open(name, "w")
    head = "QUAT,w,x,y,z,timestamp\n"
    fo.seek(0,2)
    fo.write(head)
    fo.close()
    
    fo = open(name, "a")
    #salida = np.reshape(ts_seg, (ts_seg.shape[0]*ts_seg.shape[2], ts_seg.shape[3]))
    salida = ts_seg
    
    k = 0
    for i in range(salida.shape[0]):
        if(i%128==0):
            if(i!=0):
                k+=1
        print('%d'%(i), end='\r')
        fo.write("quat0,"+str(salida[i][0])+","+str(salida[i][1])+","+str(salida[i][2])+","+str(salida[i][3])+","+str(i)+","+'\n')
        fo.write("quat1,"+str(salida[i][4])+","+str(salida[i][5])+","+str(salida[i][6])+","+str(salida[i][7])+","+str(i)+","+'\n')
        fo.write("quat2,"+str(salida[i][8])+","+str(salida[i][9])+","+str(salida[i][10])+","+str(salida[i][11])+","+str(i)+","+'\n')
        fo.write("quat3,"+str(salida[i][12])+","+str(salida[i][13])+","+str(salida[i][14])+","+str(salida[i][15])+","+str(i)+","+'\n')
        fo.write("quat4,"+str(salida[i][16])+","+str(salida[i][17])+","+str(salida[i][18])+","+str(salida[i][19])+","+str(i)+","+'\n')
        if(salida.shape[1]==36):
            fo.write("quat5,"+str(salida[i][20])+","+str(salida[i][21])+","+str(salida[i][22])+","+str(salida[i][23])+","+str(i)+","+'\n')
            fo.write("quat6,"+str(salida[i][24])+","+str(salida[i][25])+","+str(salida[i][26])+","+str(salida[i][27])+","+str(i)+","+'\n')
            fo.write("quat7,"+str(salida[i][28])+","+str(salida[i][29])+","+str(salida[i][30])+","+str(salida[i][31])+","+str(i)+","+'\n')
            fo.write("quat8,"+str(salida[i][32])+","+str(salida[i][33])+","+str(salida[i][34])+","+str(salida[i][35])+","+str(i)+","+'\n')
        else:
            fo.write("quat5,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat6,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat7,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat8,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
    fo.close()

In [53]:
name = "./00_06b_models/RESULT_GEN_1MUESTRA_00_06b.csv"
write(name, predict)